In [11]:
# Essential Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting Style Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6) # Default size for figures

# =================================================================
# Data Loading
# =================================================================
# Professional Note: The file path points directly to the 'raw/' directory
# to maintain traceability and immutability of the original data.
try:
    df_train = pd.read_csv('data/raw/train.csv')
    df_test = pd.read_csv('data/raw/test.csv')
    print("Data successfully loaded.")
except FileNotFoundError:
    print("Error: Ensure that 'train.csv' and 'test.csv' files are in the 'data/raw/' folder.")

# Set 'Id' as the index for both DataFrames
df_train = df_train.set_index('Id')
df_test = df_test.set_index('Id')

Data successfully loaded.


In [12]:
# Show the first 5 rows of the training set
print("First rows of the training set (Train Data Head):\n")
print(df_train.head())

# Review dimensions and data types
print("\nDataFrame Information (Shape and Info):\n")
print(f"Train set rows and columns: {df_train.shape}")
print(f"Test set rows and columns: {df_test.shape}")

# Inspection of data types and non-null values
df_train.info()

First rows of the training set (Train Data Head):

    MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                    
1           60       RL         65.0     8450   Pave   NaN      Reg   
2           20       RL         80.0     9600   Pave   NaN      Reg   
3           60       RL         68.0    11250   Pave   NaN      IR1   
4           70       RL         60.0     9550   Pave   NaN      IR1   
5           60       RL         84.0    14260   Pave   NaN      IR1   

   LandContour Utilities LotConfig  ... PoolArea PoolQC Fence MiscFeature  \
Id                                  ...                                     
1          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
2          Lvl    AllPub       FR2  ...        0    NaN   NaN         NaN   
3          Lvl    AllPub    Inside  ...        0    NaN   NaN         NaN   
4          Lvl    AllPub    Corner  ...        0    NaN   NaN     